# Livrable 1 : Classification Binaire
|Erwan Martin|Thibaut Liger-Hellard|Arnaud Maturel|Guillaume Le Cocguen|Victorien Goudeau|
|------------|---------------------|--------------|--------------------|-----------------|

**Contexte**:
L'objectif de ce livrable est classifier les photos des "non-photos" parmis les images du dataset.
La classification sera donc binaire.


# import des librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

import tensorflow as tf
import tensorboard


In [2]:
#verify GPU available
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Préparation des données
Afin de faire la classification binaire des images il va falloir préparer nos données en étiquettant nos images et en les mettant a la même échelle (scaling).

In [11]:
#data path definition
f = open('../data/path.txt', "r")
DATAPATH = f.read()
print(DATAPATH)

D:\CESI\A5\datascience\Projet\data
